# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

Some interesting references I don't want to lose:
http://deeplearning.net/software/theano_versions/dev/tutorial/conv_arithmetic.html
http://cs231n.github.io/convolutional-networks/
https://www.tensorflow.org/versions/r0.11/api_docs/python/
https://www.deeplearningbook.org/

https://campushippo.com/lessons/build-a-tensorflow-traffic-sign-classifier-at-95-accuracy-214727f24

---
## Step -1: Load libraries

In [ ]:
import pickle
import matplotlib.pyplot as plt
import random
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

# Visualizations will be shown in the notebook.
%matplotlib inline

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
training_file = './traffic-signs-data/train.p'
testing_file = './traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [ ]:
### Replace each question mark with the appropriate value.

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = 43

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

---
## Create Validation set
Split the training set into Training and Validation sets with 20% of the samples being held back from the training set to be used in the validaiton set.

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size = 0.2, random_state=0)

print("Updated image Shape: {}".format(X_train[0].shape))
print("Updated Training Set:   {} samples".format(len(X_train)))
print("Updated Validation Set:   {} samples".format(len(X_validation)))
print("Updated Test Set:       {} samples".format(len(X_test)))

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

# Data Exploration and Visualization
---
### STUDENT  REMARK:
<font color='red'>
Start by looking at the distribution of data samples.
</font>

In [ ]:
plt.hist(y_train, bins=n_classes)  # plt.hist passes it's arguments to np.histogram
plt.title("Histogram of sign indices in training set")
plt.show()

### STUDENT  REMARK:
<font color='red'>
In the histogram view we can see the dataset is not evenly distributed over the number of signs.  The likely effect (My hypothesis) is that the signs with more training opportunities will have a higher accuracy after training.
</font>

---
<font color='red'>
Let's also look at a few of the images to see how well/poorly they are captured.
</font>

In [ ]:
import matplotlib.gridspec as gridspec

gs_row = 2
gs_col = 10

fig = plt.figure()
gs = gridspec.GridSpec(gs_row, gs_col, wspace=0.0, hspace=0.0)
#gs.tight_layout(fig, rect=[0.45, None, None, None])
#gs.update(left=0.05, hspace=0.05, wspace=0.05)

for row in range(0,gs_row):
    for col in range(0, gs_col):

        index = random.randint(0, len(X_train))
        image = X_train[index].squeeze()
        ax = plt.subplot(gs[row, col])
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel(y_train[index])
        plt.imshow(image)

### STUDENT  REMARK:
<font color='red'>
From the small sampling of images, one can see that the lighting is different each image and some are extremely dark.  We will want to equalize that before training.
</font>

---
<font color='red'>
If there is time at the end of this assignment, it would be interesting to create a confusion matrix, like shown here: http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset#resultanalysis, to analyze the classification errors.
</font>

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

## Preprocess the data

In [ ]:
def NoName(data):
    imgs = np.ndarray((data.shape[0], 32, 32, 4), dtype=np.uint8)
    for i, img in enumerate(data):
        rows,cols = img.shape
        M = cv2.getRotationMatrix2D((cols/2,rows/2),10,1)
        dst = cv2.warpAffine(img,M,(cols,rows))
        imgs[i] = dst

    return imgs

def EqulLumAndGray(data):
    imgs = np.ndarray((data.shape[0], 32, 32, 4), dtype=np.uint8)
    for i, img in enumerate(data):    
        ycrcb=cv2.cvtColor(img,cv2.COLOR_RGB2YCR_CB)
        channels=cv2.split(ycrcb)
        cv2.equalizeHist(channels[0],channels[0])
        cv2.merge(channels,ycrcb)
        cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2RGB,img)        

        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)        
        gray_img =  np.reshape(gray_img,(32,32,1))
        cv2.equalizeHist(gray_img,gray_img)       
        myvar = np.concatenate((img, gray_img), axis=2)
#        print(myvar.shape)        
    
        imgs[i] = myvar
    return imgs


X_train = EqulLumAndGray(X_train)
X_validation = EqulLumAndGray(X_validation)
X_test = EqulLumAndGray(X_test)

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**
<font color='red'>
I tried a few approaches but finally made the decision to convert the image to YCrCb space and equalize the Y, luma, channel of the images.  The purpose of the equalization is to make the darker images lighter.<br>
<br>
In the same function, I make a grayscale version of the image, equalize it, then concatenate it to the original image as a fourth channel.  My first thought was to use only the grayscale image but then I thought that getting rid of the color channels would reduce the amount of information and it could be better to let the NN optimize weights given all of the data available.
</font>

In [ ]:
index = random.randint(0, len(X_train))
image = X_train[index].squeeze()

plt.figure(figsize=(1,1))
#plt.imshow(image, cmap='gray')
plt.imshow(image)
print(y_train[index])

In [ ]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**
<font color='red'>
I have augmented the Training data by applying random transforms to the images that are underrepresented in the set.<br>
I split out 20% of the training data for Validation<br>
I use the Testing data given.
</font>

VGG-style convnet

    Input: (32, 32, 1)
    Conv: (32, 32, 64)
    Pool: (16, 16, 64)
    Conv: (16, 16, 128)
    Pool: (8, 8, 128)
    Conv: (8, 8, 256)
    Conv: (8, 8, 256)
    Pool: (4, 4, 256)
    Flatten: (2048)
    FullyConnected: (400)
    Dropout 0.5
    FullyConnected: (200)
    Dropout 0.5
    Output: (43)

---
# Design the Network

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
from tensorflow.contrib.layers import flatten

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.0, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def LeNet(x):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    
    # A way to get 32x32x1?  **=> Achieved 0.90 accuracy on Validation set <=**
    #C0_W = weight_variable([1,1,3,1])
    #C0_b = bias_variable([1])
    #C0 = conv2d(x, C0_W)+C0_b
    
    # TODO: Layer 1: Convolutional. Input = 32x32x4. Output = 28x28x6.
    # The shape of the filter weight is (height, width, input_depth, output_depth)
    C1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 4, 6), mean=mu, stddev=sigma), name="C1_W")
    C1_b = tf.Variable(tf.zeros(6), name="C1_b") # The shape of the filter bias is (output_depth,)
    C1_strides = [1,1,1,1] # Set the stride for each dimension (batch_size, height, width, depth)
    C1_padding = 'VALID'
    C1 = tf.nn.conv2d(x,
                      C1_W,
                      C1_strides,
                      C1_padding) + C1_b

    # TODO: Activation.
    #C1 = tf.nn.sigmoid(C1)
    C1 = tf.nn.relu(C1)

    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    #C1 = tf.nn.avg_pool(C1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    C1 = max_pool_2x2(C1)
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    C2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean=mu, stddev=sigma), name="C2_W")
    C2_b = tf.Variable(tf.zeros(16), name="C2_b") # The shape of the filter bias is (output_depth,)
    C2_strides = [1,1,1,1] # Set the stride for each dimension (batch_size, height, width, depth)
    C2_padding = 'VALID'
    C2 = tf.nn.conv2d(C1,
                      C2_W,
                      C2_strides,
                      C2_padding) + C2_b
    
    # TODO: Activation.
    #C2 = tf.nn.sigmoid(C2)
    C2 = tf.nn.relu(C2)
    
    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    #C2 = tf.nn.avg_pool(C2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    C2 = max_pool_2x2(C2)
    
    # TODO: Flatten. Input = 5x5x16. Output = 400.
    C2F = flatten(C2)

    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    FC3_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma), name="FC3_W")
    FC3_b = tf.Variable(tf.zeros(120), name="FC3_b")
    FC3 = tf.matmul(C2F, FC3_W) + FC3_b
    
    # SOLUTION: Activation.
    #FC3 = tf.nn.sigmoid(FC3)
    FC3 = tf.nn.relu(FC3)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    FC4_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma), name="FC4_W")
    FC4_b  = tf.Variable(tf.zeros(84), name="FC4_b")
    FC4    = tf.matmul(FC3, FC4_W) + FC4_b
    
    # SOLUTION: Activation.
    #FC4    = tf.nn.sigmoid(FC4)
    FC4    = tf.nn.relu(FC4)
    
    #Add DROPOUT
    h_fc1_drop = tf.nn.dropout(FC4, keep_prob)

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 43.
    FC5_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma), name="FC5_W")
    FC5_b  = tf.Variable(tf.zeros(43), name="FC5_b")
    logits = tf.matmul(h_fc1_drop, FC5_W) + FC5_b

    return logits

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**
<font color='red'>
I started with leNet.<br>
ELABORATE
</font>

---
# Setup Training

In [ ]:
### Train your model here.
### Feel free to use as many code cells as needed.
x = tf.placeholder(tf.float32, (None, 32, 32, 4))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)

keep_prob = tf.placeholder(tf.float32)

#rate = 0.001

global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.001
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100000, 0.96, staircase=True)

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)

#optimizer = tf.train.AdamOptimizer(learning_rate = rate)
#training_operation = optimizer.minimize(loss_operation)

optimizer = tf.train.AdamOptimizer(learning_rate)
training_operation = optimizer.minimize(loss_operation, global_step=global_step)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, '.\lenet')
    print("Model saved")

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.